# EC2 접속 방법 - 고급
- VPC 엔드포인트 활용


## Systems Manager Session Manager
- 연결 방법 : Systems Manager Session Manager
- 동작 방식 : AWS API를 기반으로 EC2와 통신
- 연결 인증 방식 : IAM 인증 (아이디 비밀번호 필요 없음)
- 감사 방법 : 연결 기록 및 세션 사용 기록을 CloudTrail / CloudWatch로 확인 가능
- 연결 요구 사항 : SSM 에이전트 설치 + 443 포트 활성화(Outbound) / Private 인스턴스의 경우 Endpoint 설정 필요
- 주요 기능 : 기본적인 SSH 통신 + SCP 터널링 (파일 전송) / 기타 EC2에 명령 전송 (예 : Lambda에서 EC2로 명령 전송 등)
- 유료

## EC2 Instance Connect Endpoint
- 연결 방법 : SSH 혹은 RDP
- 동작 방식 : Endpoint 기반으로 EC2에 SSH/RDP 접속
- 연결 인증 방식 : IAM 인증
- 감사 방법 : 연결 기록만 감사 가능 (CloudTrail)
- 연결 요구 사항 : EC2 Instance Connect Endpoint를 특정 서브넷에 프로비전, 보안 그룹 설정 필요, 특정 인스턴스 타입 미지원(G1등)
- 주요 기능 : Private EC2 인스턴스에 베스쳔 호스트 없이 접속, 터널 생성
- 무료(계정당 최대 5개, VPC당 한 개, 서브넷 당 한 개)

### DEMO - SSM Session Manager를 통해 Private EC2 연결
- 요구 사항
    - EC2 Instance에 SSM Agent가 설치되어 있을 것 (아마존 리눅스에는 기본 설치)
    - EC2 AmazonEC2RoleforSSM Managed Policy가 포함된 역할이 적용돼 있을 것
    - VPC Endpoint가 해당 VPC에 있을 것
        - ssm, ssmmassages, ec2messages, logs, s3
    - SG에서 아웃바운드 443 포트가 열려있을 것

### DEMO - Instance Connect Endpoint를 활용한 Private EC2 연결
- EIC Endpoint에 접근 가능할 것
- 보안 그룹에 22번 포트가 열려 있을 것

### SSM Session Manager 실습 순서
1. 도쿄 리전
2. VPC 생성 (이전 강의 참고)
3. IAM 역할 만들기
    - 사용 사례 : EC2 role for~ 선택
    - 정책 추가 : cloudwatchagentserverpolicy
4. 인스턴스 프로비전
    - KP 없이 생성
    - VPC, Private Subnet, default SG
    - 인스턴스 프로파일 적용
5. VPC Endpoint들 생성
    - ssm endpoint(맨 위에꺼), 가용영역 private 선택
    - ssmmessages, 가용영역 private 선택
    - ec2 messages
    - logs
6. 세션 시작하기
    - aws systems manager의 세션관리자 메뉴, 세션 시작
7. CloudWatch 로그 그룹 만들기
    - 기본으로 생성
8. 세션 관리자 기본 설정 편집
    - CloudWatch Logging Enable 체크
    - Enforce encryption 체크 해제
9. 세션 다시 시작
    - 이거 저거 입력해보고 로그 그룹으로 이동, 로그 확인
10. 리소스 정리
    - 세션은 닫으면 됨
    - 엔드포인트 모두 삭제



### Instacne Connect Endpoint 실습 순서
1. VPC Endpoint 생성
    - 서비스 범주 : EC2 인스턴스 연결 엔드포인트
    - VPC, Default SG, Private Subnet 선택
2. 인스턴스 연결에서 엔드포인트로 연결
3. 리소스 정리
    - 인스턴스
